In [78]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np

dose = [38,40,40,40,42,42,42,44,44,44,47,47,47]
alive = [20,20,20,20,19,14,15,5,3,2,0,0,0]
dead = [0,0,0,0,1,6,5,15,17,18,20,20,20]

data_for_df = []
for i, j in zip(dose, alive):
    for n in range(j):
        data_for_df.append((i,1))
for i, j in zip(dose, dead):
    for n in range(j):
        data_for_df.append((i,0))  
df = pd.DataFrame(columns=['dose', 'Survived'])
df[['dose', 'Survived']] = data_for_df
df

,dose,Survived
0,38,1
1,38,1
2,38,1
3,38,1
4,38,1
...,...,...
255,47,0
256,47,0
257,47,0
258,47,0


In [79]:
model = smf.glm(formula="Survived ~ dose", data=df, family=sm.families.Binomial())
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               Survived   No. Observations:                  260
Model:                            GLM   Df Residuals:                      258
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -57.791
Date:                Sat, 13 Aug 2022   Deviance:                       115.58
Time:                        18:25:42   Pearson chi2:                     130.
No. Iterations:                     7   Pseudo R-squ. (CS):             0.6086
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     69.1126      9.019      7.663      0.000      51.436      86.789
dose          -1.6094      0.210     -7.653      0.000      -2.022      -1.197
==============================================================================
"""

$$x_p = \frac{logit(p)-\beta_{0}}{\beta_{1}}=\frac{logit(p)-Intercept}{coef_{dose}}$$
$$dose_{0.5} = \frac{ln(\frac{0.5}{1-0.5})-259.4485}{-69.0115} = 42.9267$$

https://bookdown.org/dereksonderegger/571/11-binomial-regression.html#prediction-and-effective-dose-levels

In [100]:
def logit(p):
    return np.log(p/(1-p))

def invlogit(x):
    # inverse function of logit
    return 1/(1+np.exp(-x))

dose = (logit(0.5)-result.params.Intercept)/result.params['dose'] # 42.941936818804464
1/result.params['dose']

-0.6213326565715561

In [109]:
# SE error

#https://stats.stackexchange.com/questions/483362/pointwise-standard-errors-for-a-logistic-regression-fit-with-statsmodels
#https://github.com/cran/MASS/blob/master/R/dose.p.R
#https://lukemiller.org/index.php/2010/02/calculating-lt50-median-lethal-temperature-aka-ld50-quickly-in-r/
def dose_p(model, p=0.5):
    logit = lambda p: np.log(p/(1-p))
    dose = (logit(p)-result.params.Intercept)/result.params['dose']
    C = result.normalized_cov_params
    L_T = np.array([1/result.params['dose'], dose/result.params['dose']])
    # Compute the pointwise standard error, as outlined above
    L_T = np.matrix(L_T)
    L = np.transpose(L_T)
    C = np.matrix(C)

    var_f = np.matmul(np.matmul(L_T, C), L)
    var_f = np.asarray(var_f)[0][0]
    pointwise_se = np.sqrt(var_f) 
#     print(pointwise_se)
    return dose, pointwise_se

In [ ]:
#https://web.pdx.edu/~joel8/resources/ConceptualPresentationResources/LogisticRegression_DoseResponse.pdf
#https://pubmed.ncbi.nlm.nih.gov/33909080/

In [132]:
dose_p(result, 1-0.001455)

(38.88383099017503, 0.5416646661584501)

In [131]:
result.predict({'dose':47})

0    0.001455
dtype: float64

Стоит отметить, что остаточное отклонение здесь (8,309) меньше, чем наши остаточные степени свободы (11), что означает, что наши данные не чрезмерно рассеяны. Если остаточное отклонение было чем-то большим, например, 20 по сравнению с нашими 11 остаточными степенями свободы, возможно, стоит попробовать логарифмическое преобразование значений обработки. Это может иметь место, если вы вводили дозы какого-либо препарата в очень широком диапазоне концентраций. Чтобы выполнить логистическую регрессию на наших логарифмически преобразованных данных, мы просто немного изменим вызов glm:

In [ ]:
# PROBIT

In [ ]:
#https://www.linkedin.com/pulse/simple-lc50-calculation-pesticide-toxicity-bioassay-using-adhiwibawa/